# Trajectory data mining in Python

In this tutorial, we will learn how to conduct exploratory space-time data analysis (ESTDA) based on movement data. When analyzing mobility data, using various visualization approaches (visual analytics) is important to be able to understand the data and extract insights from it. Here, we will learn a few tricks how we can manipulate movement data and visualize it using static and interactive visualizations in Python. 

```{admonition} Attribution

This tutorial is partly based on excellent resources provided by Anita Graser and the documentation of `movingpandas` [library](https://github.com/anitagraser/movingpandas).

```

In [ ]:
import pandas as pd
from keplergl import KeplerGl
import geopandas as gpd
import movingpandas as mpd
from datetime import datetime, timedelta
import plotly.express as px
import warnings
warnings.simplefilter("ignore")

## Input data

In this tutorial, we will use AIS data that describes vessel movements ([obtained from movingpandas](https://github.com/anitagraser/movingpandas-examples/tree/main/data)). The AIS data has been published by the Danish Maritime Authority. The AIS record sample extracted for this tutorial covers vessel traffic on the 5th July 2017 near Gothenburg. Let's start by reading it and checking how the data looks like: 

In [ ]:
# Filepath
fp = "data/ais.gpkg"
data = gpd.read_file(fp);
data.plot(figsize=(12,12), markersize=0.7, alpha=0.7)

In [ ]:
data.shape

In [ ]:
# Check first rows
data.head()

Okay as we can see, we have around 80,000 recorded observations and information about the timestamp, geometry, ship type etc. `SOG` column contains information about speed over ground. Let's take a look at the data distribution:

In [ ]:
data['SOG'].hist(bins=100, figsize=(15,3))

As we can see, most of the observations are actually such where the vessels have not been moving, i.e. they are staying at the harbor. It is not useful to keep such records in our data, so let's get rid off those: 

In [ ]:
data = data[data.SOG>0].copy()
data.shape

Okay, now we have much less observations. Let's at this point calculate `x` and `y` coordinates based on our Point geometries, so that we can easily plot our data in 3D:

In [ ]:
# Calculate x and y coordinates
data["x"] = data["geometry"].x
data["y"] = data["geometry"].y
data.head()

## Constructing trajectories

Next, we want to convert our individual point observations into **trajectories**. This can be done easily using the ``TrajectoryCollection`` functionality / class provided by movingpandas (see [docs](https://movingpandas.readthedocs.io/en/latest/trajectorycollection.html)). To be able to use this functionality, we first need to parse a ``DateTime`` index based on the ``Timestamp`` column, and that the `MMSI` number (*Maritime Mobile Service Idenity*) is used as the unique id for a ship. We also specify that the minimum length for a trajectory needs to be at least 100 meters:

In [ ]:
# Create trajectories
data['time'] = pd.to_datetime(data['Timestamp'], format='%d/%m/%Y %H:%M:%S')
data = data.set_index('time')

# Specify minimum length for a trajectory (in meters)
minimum_length = 100 
collection = mpd.TrajectoryCollection(data, 'MMSI', min_length=minimum_length)

# What did we get?
collection

As a result, we get an object that represents the collection of trajectories identified by movingpandas. We can plot all the trajectories easily by:

In [ ]:
# Plot all trajectories
collection.plot()

We can access each of these trajectories individually by looping over our collection. We can also easily calculate the speed and add information about the direction of movement to our data by:

In [ ]:
for trajectory in collection.trajectories:
    # Calculate speed
    trajectory.add_speed(overwrite=True)
    
    # Determine direction of movement
    trajectory.add_direction(overwrite=True)
    
    # Let's first check only the first trajectory (i.e. stop iteration after first run)
    break

Let's see what we got. We can access the DataFrame of the given trajectory by:

In [ ]:
trajectory.df.head()

As we can see, now we have new columns `speed` and `direction` added to the data which contain information about the speed and direction. We can plot the single trajectory using geopandas:

In [ ]:
trajectory.plot()

This kind of individual trajectory is not very informative as such. We can add more context to our visualization by adding a background map and making it interactive. Movingpandas provides an easy-to-use api to plot the trajectory on top of OSM basemap with Holoviews/Bokeh:

In [ ]:
trajectory.hvplot(frame_height=400, frame_width=400)

This map provides much more contextual information and makes it possible to understand that the trajectory is a ship which is approaching or leaving the harbor of Gothenburg (tricky to say at this point). 

## Exploratory analysis with Space Time Cube (STC)

It is also fairly easy to visualize our trajectory in 3D using space time cube, in which the 3rd dimension is based on time. There are different libraries that can be used for plotting data in 3D, but we will use here [plotly](https://plotly.com/python/plotly-express/) library which provides a relatively easy and straightforward API to generate 3D visualizations. For plotting data in 3D Let's first modify our datetime information a bit, and only keep the time because all of our values are from the same date (5th of July, 2017): 

In [ ]:
# Extract the time component
trajectory.df["time"] = pd.to_datetime(trajectory.df["Timestamp"]).dt.strftime("%H:%M")

# Plot the trajectory in space-time cube
fig = px.line_3d(trajectory.df, x="x", y="y", z="time", color='MMSI')
fig.update_traces(line=dict(width=5))

# Want to get background map as well? It is possible:
# https://chart-studio.plotly.com/~empet/14397/plotly-plot-of-a-map-from-data-available/#/

fig.show()

Great, now we can see our trajectory in 3D! Based on this viusalization we can already understand our data a bit better. We can for example, see how the ship has slowly approached the harbor and then docked around 18:00 when the movement has stopped (i.e. x/y coordinates do not change anymore). In a similar manner, we can process all our trajectories and plot them all together: 

In [ ]:
trajectories = gpd.GeoDataFrame()

i = 0
for trajectory in collection.trajectories:
    # Calculate speed
    trajectory.add_speed(overwrite=True)
    
    # Determine direction of movement
    trajectory.add_direction(overwrite=True)
    
    # Parse time component (as datetime)
    trajectory.df["t"] = pd.to_datetime(trajectory.df["Timestamp"])
    
    # Add unique id for each trajectory
    trajectory.df["tid"] = i
    
    # Add to container
    trajectories = trajectories.append(trajectory.df)
    i+=1
    
# Reorder the data based on timestamp
trajectories = trajectories.sort_values(by="t").reset_index(drop=True)

# Plot them all (use trajectory id to separate trajectories)
fig = px.line_3d(trajectories, x="x", y="y", z="t", color='MMSI')
fig.update_traces(line=dict(width=5))

# Want to get background map as well? It is possible:
# https://chart-studio.plotly.com/~empet/14397/plotly-plot-of-a-map-from-data-available/#/

fig.show()

The 3D visualization of multiple trajectories can look a bit messy (a bit like spaghetty), but we can already identify some patterns from the spate-time data. Looking at how the trajectories cluster in the STC, it is evident that there is a lot of activity throughout the day. There are two clear clusters in the data which basically represent activity related to two harbors in the area (Gothenburgs harbor and Saltholmens Brygga). From the data you can see how ships and boats are steadily approaching the harbors using more or less the same routes (at different times). Looking at the data, we can also see that one ship stayed the whole day in the harbor without moving anywhere (vertical line without movement). 

## Aggregating trajectories

As we can saw from the previous examples, looking at the movement data in 3D using STC can provide interesting insights that can be tricky to understand otherwise. However, understanding and reading the information takes a bit of time and experience before you start to understand what is going on in the data. There are however more tricks that we can do, to abstract our data and make it easier to understand in 2D. Movingpandas provides a handy tool called ``TrajectoryCollectionAggregator`` which can be used to aggregate the trajectories by extracting clusters of significant trajectory points and computing flows between the clusters. These significant points of trajectories include e.g. their start and end points, the points of significant turns, and the points of significant stops, i.e. pauses in the movement (Andrienko & Andrienko, 2011). Once the points have been detected, then they are grouped based on their spatial proximity and flows are generated between them. We can aggregate the movement data following this approach easily with movingpandas, by specifying minimum and maximum threshold distances between significant points and minimum duration (in seconds) required to detect a stop from the data:

In [ ]:
# Aggregate trajectories
# Method description: https://movingpandas.readthedocs.io/en/latest/trajectoryaggregator.html
agg = mpd.TrajectoryCollectionAggregator(collection, min_distance=500, max_distance=2000, min_stop_duration=timedelta(minutes=10))

# Get flows and clusters
flows = agg.get_flows_gdf()
clusters = agg.get_clusters_gdf()

As a result, we get DataFrames having information about weight of the flows as well as the size of the cluster:

In [ ]:
flows.head()

In [ ]:
clusters.head()

Based on this information, we can create an interactive map that provides more easily understandable view for our spatio-temporal movement data:

In [ ]:
# Plot with Holoviews
( flows.hvplot(title='Generalized aggregated trajectories', geo=True, hover_cols=['weight'], line_width='weight', alpha=0.5, color='#1f77b3', tiles='OSM', frame_height=400, frame_width=400) * 
  clusters.hvplot(geo=True, color='red', size='n') )

Great! As a result, we have a map which is quite easy to interpret with width of the line representing the volume of vessels moving between specific locations. We were able to get a sense of these patterns by looking at the movement data in 3D (i.e. where lot's of movement happens), but following this aggregation approach and visualizing the movement flows makes it much easier to get a grasp of the data. 

## Animating and exploring the trajectory characteristics in 3D 

The previous approaches have provided useful ways to extract information from movement data. Next, we will see how we can extract further characteristics based on the data using a WebGL based interactive visualization tool developed by Uber: [Kepler.gl](https://kepler.gl/).

Let's first manipulate our movement data a bit so that we can visualize it easily in 3D using Kepler.gl. Basically, what we want to do is to generate line segments between all observations in our data and convert them into Polygons by making a small buffer around them. This trick is needed to be able to take advantage of the full 3D capabilities of Kepler.gl. Let's start by preparing a couple of helper functions that generates the segments and converts them into a Polygon:

In [ ]:
from shapely.geometry import LineString

def to_linestring(row):
    if row["end_geometry"] is not None:
        return LineString([row["geometry"], row["end_geometry"]])

def prepare_segments(trajectory):
    # Calculate segments for a single trajectory
    df = trajectory.df.join((trajectory.df["geometry"].shift(-1).to_frame().rename(columns={'geometry': 'end_geometry'})))

    # Create linestring
    df["segment"] = df[["geometry", "end_geometry"]].apply(to_linestring, axis=1)

    # Update geometry column with segments
    df["geometry"] = df["segment"]

    # Drop empty geoms (there is one after converting from points to lines)
    df = df.dropna(subset=["geometry"])

    # Drop unnecessary columns
    df = df.drop(["segment", "end_geometry"], axis=1)

    # Convert lines to polygons, so that volume/height can be adjusted
    df["geometry"] = df.buffer(0.0002)
    return df

In [ ]:
movements = gpd.GeoDataFrame()

for trajectory in collection.trajectories:
    
    # Parse direction of movement
    trajectory.add_direction(overwrite=True)

    # Parse speed
    trajectory.add_speed(overwrite=True)

    # Create segments and calculate some useful statistics
    segments = prepare_segments(trajectory)
    movements = movements.append(segments)
    
    # Increase index
    i+=1
    
# Reset index
movements = movements.reset_index(drop=True)

# Drop "time" column which is in datetime format (Kepler.gl don't like it when saving the output)
movements = movements.drop(["time"], axis=1)

# What did we get?
movements.head()

Okay, now we have prepared our data and we can visualize it with Kepler.gl. What we will do next is to:

1. Determine the attribute which is used for coloring the trajectories (name)
2. Determine that the height of data is based on the speed so that we can easily see areas and segments where the ships/boats have been moving fast
3. Add a filter based on timestamp
4. Add another filter based on ship type
5. Animate the movements with a selected time window

Notice, that these will be done using the graphical interface of Kepler.gl, hence, you need to check the video to understand how to repeat the steps. Initializing the Kepler.gl and adding our movement data to it is easy:

In [ ]:
# Create a basemap 
m = KeplerGl(height=800, width=800)

# Add the movement data
m.add_data(movements, "trajectories")

# Render the map / interface
m

Once, we have prepared our visualization in a way how we want it, we can easily save our map into a dedicated html website with the current state of the page by:

In [ ]:
m.save_to_html(file_name='Trajectories_3D.html')